# milestone-II-stock-predictions
This code is being produced to fulfil the requirements of the Milestone 2 project in the University of Michigan Masters in Applied Data Science degree.

This Milestone project will build on the Milestone 1 Project. The goal of this project will be to study stock market data and predict future returns. We will extend the data, data manipulation processes and feature generation approach from that project to create the dataset for this project. We will also explore unsupervised learning to attempt to get a better understanding of the categories of stocks included in this project, and their changes over time. One output of the unsupervised learning we conduct will be to contribute additional features to the supervised learning approach. Our dataset will include approximately 250-300 UK stocks over approximately five years from 2016 to 2021.

## supervised learning
The input data consists of 3 types of features. They are 1) stock price histories including amount traded per day (volume), 2) fundamental data about the company such as revenue, earnings per share, and 3) technical features derived from price histories and volume. Many of these are Finance-domain specific but others are more widely applicable across domains. In total there are over 1000 features. This notebook focuses on supervised learning approaches to this problem. Through analysis with this dataset we have found that model performance using all or most of the features did not generalize well to the test set. Therefore I focused on a recursive feature selection approach to select a subset of features that would perform better on both the train and test sets. This recursive feature selection approach is coded bespoke. It allows the possibility to add or remove a feature from the current best set. If the result is better with the new featureset that becomes the new best featureset. Since predicting future stock market returns is a time-series problem we have kept two validation data sets, one is concurrent with the test set and another will be for a time period after the test set. In this notebook and the recursive feature selection we experimented with many different models and found that the best results by quite a margin were achieved with Random Forest models. We predominantly intend to treat this as a regression problem attempting to predict the percentage price change from the date of the featureset to the date of the future price. However, the notebook is setup with the option to treat this as a classification problem instead. In that case, we would be predicting whether the stock will go up over the next 20 trading days.

The dataset contains daily stock information for a set of 253 UK stocks covering dates from January 2017 to September 2021. If an approach like this was used to inform trading decisions, it would make sense to focus only on the stocks that are predicted to go up the most. If this is run as a classification problem then it would make sense to focus on stocks that are predicted to have the highest probability of going up. In either case an appropriate evaluation metric is precision. Just as in trading, we don't have to trade every stock but can focus on the stocks we think are worth trading, so with precision we can evaluate only on stocks we think will be the best performers over the 20 trading day time horizon selected. We will do evaluation in 2 ways, firstly, precision for stocks that are predicted to go up and precision for the top x% of predictions. I would hope and expect that the top x% of predictions will achieve a higher precision than the precision for all stocks predicted to go up. Of course in trading a key consideration isn't just whether stocks go up but how much they go up or down. Our second approach to evaluation will be to measure the average return of all stocks that are predicted to go up. We will also measure the average return of the top x% of stocks. Again we would hope and aim to see the average return for the top x% of stocks being higher than the average return of all stocks predicted to go up.

As a naive benchmark we will calculate precision and average return in the case that we simply and unrealistically predict that all stocks will go up every day. Since the signal to noise ratio in Finance is low, this benchmark may not be trivial to outperform.

## unsupervised learning
This project will also incorporate some analysis through unsupervised learning. Most of that will be done in a separate notebook. However, in this notebook and the associated utilities file we run pca to add to the set of features that is considered during the recursive feature selection approach.

In [1]:
import numpy as np
import pandas as pd
from sklearn.linear_model import BayesianRidge, LinearRegression, ElasticNet, Ridge, Lasso, LogisticRegression, PassiveAggressiveRegressor, Lars
from sklearn import tree
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_absolute_error, precision_score
from matplotlib import pyplot as plt
from tqdm import tqdm
import os
import copy

from utils_milestone2 import get_numeric_non_infinite_cols, add_pca_cols, scale_train_test, pca_train_test, run_model, \
feature_engineering, threshold_precision, full_and_threshold_scoring, pred_to_clf_pred, load_data, \
generate_benchmark_scores, return_results, data_pipeline

### Parameters

In [2]:
Run_Feature_Selection = False
model_type="regression"

### Load the Data

In [3]:
%%time
X_train_numeric_plus_pca, X_test_numeric_plus_pca, X_val_numeric_plus_pca, X_val_final_numeric_plus_pca, y_train, y_test, y_val, y_val_final = data_pipeline(return_y_val=True)

C:\Users\micha\mads\milestone-II-stock-predictions\utils_milestone2.py:76: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Volume_over_Volume_MA50'] = df['Volume'] / df['Volume_MA50']
C:\Users\micha\mads\milestone-II-stock-predictions\utils_milestone2.py:77: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Volume_over_Volume_MA200'] = df['Volume'] / df['Volume_MA200']
C:\Users\micha\mads\milestone-II-stock-predictions\utils_milestone2.py:78: SettingWithCopyWarning: 
A value is trying to be set on a copy 

Total Explained 0.9992301541002819
Wall time: 35.7 s


### Prepare the target data to handle classification and regression scoring
Classify train examples as 1 if the price went up otherwise 0.

In [4]:
y_train_clf = pred_to_clf_pred(y_train, threshold=1)
y_test_clf = pred_to_clf_pred(y_test, threshold=1)
y_val_clf = pred_to_clf_pred(y_val, threshold=1)
y_val_final_clf = pred_to_clf_pred(y_val_final, threshold=1)

Create a naive benchmark where everything is predicted to go up.

In [5]:
y_all_up_train = [1] * len(y_train_clf)
y_all_up_test = [1] * len(y_test_clf)
y_all_up_val = [1] * len(y_val_clf)
y_all_up_val_final = [1] * len(y_val_final_clf)

### Calculate Benchmark Scores

In [6]:
benchmark_scores = generate_benchmark_scores(y_train, y_test, y_train_clf, y_all_up_train, y_test_clf, y_all_up_test)

In [7]:
print("Benchmark figures over 20 day prediction horizon:")
benchmark_scores

Benchmark figures over 20 day prediction horizon:


{'train_precision': 0.5381,
 'test_precision': 0.5835,
 'train_return': 0.005,
 'test_return': 0.0221}

This is the benchmark precision if we predict everything goes up. Now we will test the precision of our model. We can see that market conditions in the test period were noticeably different to the training period as stocks were going up a lot more frequently over 20 day trading horizons.

### Feature Selection
We saw in the earlier notebook that modeling with just a few features achieved better results than using all 1000+ features. Here we will start with a tiny number of features and iteratively try adding (& later dropping) features to improve model performance.

In [8]:
cols = X_train_numeric_plus_pca.columns.to_list()#[::-1]

In [24]:
# random forest
#selected_cols = ['SR', 'ROCE_TTM_pct', 'Price_to_FCF_TTM', 'MACDSIGNAL_55', 'trend_ema_slow', 'others_dlr', 'volume_cmf', 'DX_14', 'MINUS_DM_14', 'ROCR100_25', 'RSI_25', 'WILLR_25', 'MFI_90', 'APO_f13_s89', 'MACDEXT_macdsignal_f3_s8_sig55', 'volume_vpt', 'APO_f5_s89'] # Random Forest tuned on threshold return
#selected_cols = ['ROCE_TTM_pct', 'Price_to_FCF_TTM', 'MACDSIGNAL_55', 'trend_ema_slow', 'others_dlr'] # manual random forest selection
#selected_cols = ['SR', 'Volume_MA50', 'Volume_MA200', 'ADOSC_f2_s89', 'AROONup_40', 'AROONdown_90', 'CCI_90', 'MACDHIST_34', 'PPO_f2_s3', 'PPO_f8_s13']

# knn
selected_cols = ['ROCE_TTM_pct', 'Price_to_FCF_TTM', 'MACDSIGNAL_55', 'trend_ema_slow', 'others_dlr', 'ValueRank', 'Price_to_Book_Latest', 'EV_to_EBITDA_TTM', 'Price_vs_MA_50d_pct', 'volume_cmf', 'volatility_bbl', 'Free_Float_pct', 'Price_vs_MA_200d_pct', 'volatility_atr', 'trend_macd', 'Open', 'volatility_dcl', 'trend_macd_diff', 'RSI_25', 'MINUS_DI_40', 'ADX_14', 'ADX_40', 'PPO_f2_s21', 'APO_f13_s21', 'MACDEXT_macdsignal_f2_s8_sig21'] 
#selected_cols = ['ROCE_TTM_pct', 'Price_to_FCF_TTM', 'MACDSIGNAL_55', 'trend_ema_slow', 'others_dlr', 'ValueRank', 'Price_to_Book_Latest', 'EV_to_EBITDA_TTM', 'Price_vs_MA_50d_pct', 'volume_cmf', 'volatility_bbl', 'High', 'Free_Float_pct', 'Price_vs_MA_200d_pct', 'volatility_atr', 'trend_macd']
#selected_cols = ['pca_1', 'pca_2', 'pca_3', 'pca_4', 'pca_5', 'pca_6', 'pca_7', 'pca_8', 'pca_9', 'pca_10']

# bayesian ridge
#selected_cols = ['RSI_40', 'Open', 'EPS_Growth_TTM_pct', 'volume_fi', 'volatility_dcm', 'trend_ichimoku_a', 'momentum_tsi', 'momentum_stoch', 'momentum_stoch_signal', 'AD', 'ADOSC_f2_s8', 'ADOSC_f3_s8', 'WILLR_2', 'WILLR_40', 'PLUS_DI_90', 'MACDEXT_macdsignal_f3_s5_sig8', 'Sales_Growth_TTM_pct_NaN']

# lars
#selected_cols = ['volume_cmf', 'Open', 'Low', 'Adj. close**', 'volatility_bbm', 'volatility_bbhi', 'volatility_kcw', 'trend_mass_index', 'trend_aroon_ind', 'trend_psar_up_indicator', 'momentum_kama', 'ADOSC_f2_s34', 'DX_5', 'MINUS_DM_5', 'ROC_10', 'AROONOSC_20', 'WILLR_20', 'PLUS_DI_25', 'AROONOSC_60', 'WILLR_60', 'MACDEXT_macdhist_f5_s21_sig34', 'MACDEXT_macd_f8_s13_sig5', 'MACDEXT_macdhist_f13_s89_sig8', 'ATR_90', 'volatility_kcp']

In [10]:
default_precision_test = 0
default_precision_train = 0
default_return_test = 0
default_return_train = 0

### Model Selection and Features to Search

In [41]:
cols_to_search = copy.deepcopy(cols)
#model = RandomForestRegressor(max_depth=3, n_estimators=200, random_state=6, criterion="mse", n_jobs=-1)
#model = BayesianRidge()
#model = Lars()
model = KNeighborsRegressor(n_neighbors=5)

In [12]:
%%time

loop_best = 0
if Run_Feature_Selection:
    for a in range(20): # later loops give a chance to drop features that were useful originally but are not useful with the latest best feature set   
        for c in tqdm(cols_to_search):
            current_selection = copy.deepcopy(selected_cols)
            if c in current_selection and len(current_selection) > 1:
                current_selection = [x for x in current_selection if x!=c]#.remove(c)
            else:
                current_selection.append(c)

            train_scores, test_scores, model = run_model(model, current_selection, X_train_numeric_plus_pca, X_test_numeric_plus_pca, y_train, y_test, model_type=model_type)

            if test_scores['default_return'] > default_return_test and train_scores['default_return'] > default_return_train:
                default_return_test = test_scores['default_return']
                default_return_train = train_scores['default_return']
                selected_cols = copy.deepcopy(current_selection)

        print("train return at default", default_return_train, "test return at default", default_return_test)
        print("##################################################################")
        print(selected_cols)

        if loop_best==default_return_test:
            print("early stopping no improvement")
            break
        loop_best=default_return_test

    

Wall time: 0 ns


### Final Model Run
With our selected features we can run the final model. Note this features were selected by the recursive feature selection technique above, but, that is not illustrated in this notebook because for random forest the recursive feature selection/elimination can take many hours to run.

In [42]:
%%time
train_scores, test_scores, model = run_model(model, selected_cols, X_train_numeric_plus_pca, X_test_numeric_plus_pca, y_train, y_test, model_type=model_type, percentile=95)

Wall time: 24.8 s


In [43]:
#print_benchmarks()
print("############################################################")
print("Number of selected features:", len(selected_cols), "from", len(X_train_numeric_plus_pca.columns), "available features")
print("############################################################")
print("Results Summary")
results = return_results(benchmark_scores, train_scores, test_scores, "test")
results

############################################################
Number of selected features: 25 from 1302 available features
############################################################
Results Summary


,dataset,metric,benchmark,default,threshold
0,train,precision,0.538,0.895,0.993
1,test,precision,0.584,0.600,0.604
2,train,return,0.50%,5.56%,21.09%
3,test,return,2.21%,2.61%,4.96%


Here we can see that with the selected features, which are a very small subset of the total available features, we have achieved precision and average return for our top 5% of predictions on train and test that is considerably better than the benchmark. Where we measure precision and average return on all stocks where the prediction was that the price would go up we have also beaten the benchmark.



### Validation Run
Once we have selected our preferred model based on the metrics seen on the training and test sets we run validation. It is important not to run validation frequently, but only run it at the end of the train and test process.

In [44]:
train_scores, val_scores, model = run_model(model, selected_cols, X_train_numeric_plus_pca, X_val_numeric_plus_pca, y_train, y_val, model_type=model_type, percentile=95)
val_benchmark_scores = generate_benchmark_scores(y_train, y_val, y_train_clf, y_all_up_train, y_val_clf, y_all_up_val)
val_results = return_results(val_benchmark_scores, train_scores, val_scores, "validation")
val_results

,dataset,metric,benchmark,default,threshold
0,train,precision,0.538,0.895,0.993
1,validation,precision,0.577,0.592,0.588
2,train,return,0.50%,5.56%,21.09%
3,validation,return,2.14%,2.49%,4.24%


### Out-of-Sample Validation
As a final check we use data that has only just been retrieved based on the latest prices. This data was not available when the model training was done and therefore there can be no leaks.

In [45]:
%%time
train_scores, val_scores_final, model = run_model(model, selected_cols, X_train_numeric_plus_pca, X_val_final_numeric_plus_pca, y_train, y_val_final, model_type=model_type, percentile=95)
val_benchmark_scores_final = generate_benchmark_scores(y_train, y_val_final, y_train_clf, y_all_up_train, y_val_final_clf, y_all_up_val_final)
val_results_final = return_results(val_benchmark_scores_final, train_scores, val_scores_final, "validation final")
val_results_final

Wall time: 17.1 s


,dataset,metric,benchmark,default,threshold
0,train,precision,0.538,0.895,0.993
1,validation final,precision,0.643,0.651,0.632
2,train,return,0.50%,5.56%,21.09%
3,validation final,return,1.89%,2.01%,2.13%


Only run when model is Random Forest
## Now let's take a look at the feature importances from our Random Forest Model.

In [47]:
# features=X_train_numeric_plus_pca[selected_cols].columns
# importances = model.feature_importances_
# indices = np.argsort(importances)

# plt.figure(1);
# plt.title('Feature Importances');
# plt.barh(range(len(indices)), importances[indices], color='b', align='center');
# plt.yticks(range(len(indices)), features[indices]);
# plt.xlabel('Random Forest Feature Importance');

## 